In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score

We're ready to build our first neural network. We will have multiple features we feed into our model, each of which will go through a set of perceptron models to arrive at a response which will be trained to our output.

Like many models we've covered, this can be used as both a regression or classification model.

First, we need to load our dataset. For this example we'll use The Museum of Modern Art in New York's [public dataset](https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv) on their collection.

In [2]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [3]:
artworks.columns

Index(['Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [4]:
# Select Columns.
artworks = artworks[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks['URL'] = artworks['URL'].notnull()
artworks['ThumbnailURL'] = artworks['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks = artworks[artworks['Department']!='Film']
artworks = artworks[artworks['Department']!='Media and Performance Art']
artworks = artworks[artworks['Department']!='Fluxus Collection']

# Drop missing data.
artworks = artworks.dropna()

In [5]:
artworks.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
0,Otto Wagner,(Austrian),(Male),1896,Architecture & Design,1996-04-09,True,True,48.6000,168.9000
1,Christian de Portzamparc,(French),(Male),1987,Architecture & Design,1995-01-17,True,True,40.6401,29.8451
2,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,34.3000,31.8000
3,Bernard Tschumi,(),(Male),1980,Architecture & Design,1995-01-17,True,True,50.8000,50.8000
4,Emil Hoppe,(Austrian),(Male),1903,Architecture & Design,1997-01-15,True,True,38.4000,19.1000


## Building a Model

Now, let's see if we can use multi-layer perceptron modeling (or "MLP") to see if we can classify the department a piece should go into using everything but the department name.

Before we import MLP from SKLearn and establish the model we first have to ensure correct typing for our data and do some other cleaning.

In [6]:
# Get data types.
artworks.dtypes

Artist           object
Nationality      object
Gender           object
Date             object
Department       object
DateAcquired     object
URL                bool
ThumbnailURL       bool
Height (cm)     float64
Width (cm)      float64
dtype: object

The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [7]:
artworks['DateAcquired'] = pd.to_datetime(artworks.DateAcquired)
artworks['YearAcquired'] = artworks.DateAcquired.dt.year
artworks['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [8]:
# Remove multiple nationalities, genders, and artists.
artworks.loc[artworks['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks.loc[artworks['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks.loc[artworks['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'



In [9]:
# Convert dates to start date, cutting down number of distinct examples.
artworks['Date'] = pd.Series(artworks.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]
#what does this do {}


In [10]:
#
# Final column drops and NA drop.
X = artworks.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks.Artist)
nationalities = pd.get_dummies(artworks.Nationality)
dates = pd.get_dummies(artworks.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates], axis=1)

Y = artworks.Department

In [11]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(100,))
mlp.fit(X, Y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [12]:
mlp.score(X, Y)

0.56100427044223433

In [13]:
Y.value_counts()/len(Y)

Prints & Illustrated Books    0.522179
Photography                   0.229085
Architecture & Design         0.111557
Drawings                      0.102997
Painting & Sculpture          0.034183
Name: Department, dtype: float64

In [14]:
from sklearn.model_selection import cross_val_score
cross_val_score(mlp, X, Y, cv=5)

array([ 0.58250943,  0.6646924 ,  0.48727012,  0.46075574,  0.35151921])

Now we got a lot of information from all of this. Firstly we can see that the model seems to overfit, though there is still so remaining performance when validated with cross validation. This is a feature of neural networks that aren't given enough data for the number of features present. _Neural networks, in general, like_ a lot _of data_. You may also have noticed something also about neural networks: _they can take a_ long _time to run_. Try increasing the layer size by adding a zero. Feel free to interrupt the kernel if you don't have time...

Also note that we created bools for artist's name but left them out. Both of the above points are the reason for that. It would take much longer to run and it would be much more prone to overfitting.

## Model parameters

Now, before we move on and let you loose with some tasks to work on the model, let's go over the parameters.

We included one parameter: hidden layer size. Remember in the previous lesson, when we talked about layers in a neural network. This tells us how many and how big to make our layers. Pass in a tuple that specifies each layer's size. Our network is 1000 neurons wide and one layer. (100, 4, ) would create a network with two layers, one 100 wide and the other 4.

How many layers to include is determined by two things: computational resources and cross validation searching for convergence. It's generally less than the number of input variables you have.

You can also set an alpha. Neural networks like this use a regularization parameter that penalizes large coefficients just like we discussed in the advanced regression section. Alpha scales that penalty.

Lastly, we'll discuss the activation function. The activation function determines whether the output from an individual perceptron is binary or continuous. By default this is a 'relu', or 'rectified linear unit function' function. In the exercise we went through earlier we used this binary function, but we discussed the _sigmoid_ as a reasonable alternative. The _sigmoid_ (called 'logistic' by SKLearn because it's a 'logistic sigmoid function') allows for continuous variables between 0 and 1, which allows for a more nuanced model. It does come at the cost of increased computational complexity.

If you want to learn more about these, study [activation functions](https://en.wikipedia.org/wiki/Activation_function) and [multilayer perceptrons](https://en.wikipedia.org/wiki/Multilayer_perceptron). The [Deep Learning](http://www.deeplearningbook.org/) book referenced earlier goes into great detail on the linear algebra involved.

You could also just test the models with cross validation. Unless neural networks are your specialty cross validation should be sufficient.

For the other parameters and their defaults, check out the [MLPClassifier documentaiton](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier).

## Drill: Playing with layers

Now it's your turn. Using the space below, experiment with different hidden layer structures. You can try this on a subset of the data to improve runtime. See how things vary. See what seems to matter the most. Feel free to manipulate other parameters as well. It may also be beneficial to do some real feature selection work...

In [15]:
# Your code here. Experiment with hidden layers to build your own model.



In [16]:
artworks = pd.read_csv('https://media.githubusercontent.com/media/MuseumofModernArt/collection/master/Artworks.csv')

In [18]:
artworks2 = artworks.sample(frac = .05)

We'll also do a bit of data processing and cleaning, selecting columns of interest and converting URL's to booleans indicating whether they are present.

In [19]:
# Select Columns.
artworks2 = artworks2[['Artist', 'Nationality', 'Gender', 'Date', 'Department',
                    'DateAcquired', 'URL', 'ThumbnailURL', 'Height (cm)', 'Width (cm)']]

# Convert URL's to booleans.
artworks2['URL'] = artworks2['URL'].notnull()
artworks2['ThumbnailURL'] = artworks2['ThumbnailURL'].notnull()

# Drop films and some other tricky rows.
artworks2 = artworks2[artworks2['Department']!='Film']
artworks2 = artworks2[artworks2['Department']!='Media and Performance Art']
artworks2 = artworks2[artworks2['Department']!='Fluxus Collection']

# Drop missing data.
artworks2 = artworks2.dropna()

In [20]:
artworks2.head()

,Artist,Nationality,Gender,Date,Department,DateAcquired,URL,ThumbnailURL,Height (cm),Width (cm)
35707,Jorge Demirjian,(Argentine),(Male),1967,Drawings,1969-10-28,True,False,70.200000,50.500000
12331,Wols (A. O. Wolfgang Schulze),(German),(Male),1949,Prints & Illustrated Books,1954-04-06,True,True,6.200000,10.000000
5372,Almir Mavignier,(Brazilian),(Male),1966,Architecture & Design,1967-04-11,True,True,85.000000,59.200000
61986,Keith Sonnier,(American),(Male),1975,Prints & Illustrated Books,1975-10-07,True,False,89.100000,20.300000
121261,Jorge Castillo,(Venezuelan),(Male),1972,Architecture & Design,2014-10-06,True,False,20.320041,25.400051


The `DateAcquired` column is an object. Let's transform that to a datetime object and add a feature for just the year the artwork was acquired.

In [21]:
artworks2['DateAcquired'] = pd.to_datetime(artworks2.DateAcquired)
artworks2['YearAcquired'] = artworks2.DateAcquired.dt.year
artworks2['YearAcquired'].dtype

dtype('int64')

Great. Let's do some more miscellaneous cleaning.

In [22]:
# Remove multiple nationalities, genders, and artists.
artworks2.loc[artworks2['Gender'].str.contains('\) \('), 'Gender'] = '\(multiple_persons\)'
artworks2.loc[artworks2['Nationality'].str.contains('\) \('), 'Nationality'] = '\(multiple_nationalities\)'
artworks2.loc[artworks2['Artist'].str.contains(','), 'Artist'] = 'Multiple_Artists'



In [23]:
# Convert dates to start date, cutting down number of distinct examples.
artworks2['Date'] = pd.Series(artworks2.Date.str.extract(
    '([0-9]{4})', expand=False))[:-1]
#what does this do {}


In [24]:
#
# Final column drops and NA drop.
X = artworks2.drop(['Department', 'DateAcquired', 'Artist', 'Nationality', 'Date'], 1)

# Create dummies separately.
artists = pd.get_dummies(artworks2.Artist)
nationalities = pd.get_dummies(artworks2.Nationality)
dates = pd.get_dummies(artworks2.Date)

# Concat with other variables, but artists slows this wayyyyy down so we'll keep it out for now
X = pd.get_dummies(X, sparse=True)
X = pd.concat([X, nationalities, dates, artists], axis=1)

Y = artworks2.Department

In [25]:
# Alright! We've done our prep, let's build the model.
# Neural networks are hugely computationally intensive.
# This may take several minutes to run.

# Import the model.
from sklearn.neural_network import MLPClassifier

# Establish and fit the model, with a single, 1000 perceptron layer.
mlp = MLPClassifier(hidden_layer_sizes=(100,50,20), activation = 'logistic',alpha = .01)
mlp.fit(X, Y)

MLPClassifier(activation='logistic', alpha=0.01, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [26]:
mlp.score(X, Y)

0.53506592776609974

In [27]:
Y.value_counts()/len(Y)

Prints & Illustrated Books    0.522263
Photography                   0.228932
Architecture & Design         0.108733
Drawings                      0.102618
Painting & Sculpture          0.037455
Name: Department, dtype: float64

In [28]:

cross_val_score(mlp, X, Y, cv=5)

array([ 0.52812202,  0.52958015,  0.6217765 ,  0.53014354,  0.53735632])

In [29]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(100,50), activation = 'logistic',alpha = .01)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.781196254539
0.732061856471


In [30]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(100,50), activation = 'logistic',alpha = 1)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.527422128798
0.527036438679


In [31]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(100,50), activation = 'logistic',alpha = .02)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.672081024269
0.706660275912


In [32]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(100,50), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.789604433403
0.676720599023


Now that we found the a good alpha, lets adjust the layers and number of perceptrons. 
I went ahead an ordered them, but worst performing, to best. 

In [42]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(100,10), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.712784253774
0.530099739565


In [41]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(75,10), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.536212497611
0.66675889801


In [37]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(300,200), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.841964456335
0.668301271805


In [34]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(200,200), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.795146187655
0.65884976344


In [45]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(200,100), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.545385056373
0.641288930295


In [33]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(100,100), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.804700936365
0.683171321855


In [43]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(50,25), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.785018154023
0.673418769624


In [40]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(200,50), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.848079495509
0.71183065398


In [39]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(300,50), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.533919357921
0.725016699286


In [36]:
#best so far
mlp = MLPClassifier(hidden_layer_sizes=(75,25), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.802598891649
0.712561247962


In [44]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(200,25), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.807567360978
0.715438882341


In [35]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(150,50), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.803936556469
0.737451397043


In [38]:
#try different params
mlp = MLPClassifier(hidden_layer_sizes=(300,100), activation = 'logistic',alpha = .009)
mlp.fit(X, Y)
print(mlp.score(X,Y))
print(cross_val_score(mlp, X, Y, cv=5).mean())

0.764188801835
0.736101812474


overall, it seems like we have managed to correct for the overfitting, and improve overall accuracy using these parameters. 
There is still room for improvement, however, and could likely be achieved through some feature engineering. 